In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dropout
from sklearn.model_selection import train_test_split
import zipfile
import os
import cv2

import h5py
from six.moves import cPickle as pickle

%matplotlib inline

In [0]:
import h5py

# The DigitStructFile is just a wrapper around the h5py data.  It basically references
#    input:              The input h5 matfile
#    digitStruct_name:   The h5 ref to all the file names
#    digitStruct_bbox:   The h5 ref to all struc data

class DigitStructFile:
    def __init__(self, input):
        self.input = h5py.File(input, 'r')
        self.digitStruct_name = self.input['digitStruct']['name']
        self.digitStruct_bbox = self.input['digitStruct']['bbox']

    # get_name returns the 'name' string for the n(th) digitStruct.

    def get_name(self,n):
        return ''.join([chr(c[0]) for c in self.input[self.digitStruct_name[n][0]].value])

    def bbox_helper(self,attribute):
        if (len(attribute) > 1):
            attribute = [self.input[attribute.value[j].item()].value[0][0] for j in range(len(attribute))]
        else:
            attribute = [attribute.value[0][0]]
        return attribute

    # get_bbox returns a 'dict' of data for the n(th) bbox.
    def get_bbox(self,n):
        bbox = {}
        bb = self.digitStruct_bbox[n].item()
        bbox['height'] = self.bbox_helper(self.input[bb]["height"])
        bbox['label']  = self.bbox_helper(self.input[bb]["label"])
        bbox['left']   = self.bbox_helper(self.input[bb]["left"])
        bbox['top']    = self.bbox_helper(self.input[bb]["top"])
        bbox['width']  = self.bbox_helper(self.input[bb]["width"])
        return bbox

    def get_DigitStructure(self,n):
        s = self.get_bbox(n)
        s['name']=self.get_name(n)
        return s

    # get_all_DigitStructure returns all the digitStruct from the input file.
    def get_all_DigitStructure(self):
        return [self.get_DigitStructure(i) for i in range(len(self.digitStruct_name))]
        
# Return a restructured version of the dataset (one structure by boxed digit).
#   Return a list of such dicts :
#      'filename' : filename of the samples
#      'boxes' : list of such dicts (one by digit) :
#      'label' : 1 to 9 corresponding digits. 10 for digit '0' in image.
#      'left', 'top' : position of bounding box
#      'width', 'height' : dimension of bounding box

    def get_all_DigitStructure_ByDigit(self):
        img_data = self.get_all_DigitStructure()
        result = []
        struct_count = 1
        for i in range(len(img_data)):
            item = { 'filename' : img_data[i]["name"] }
            figures = []
            for j in range(len(img_data[i]['height'])):
                figure = {}
                figure['height'] = img_data[i]['height'][j]
                figure['label']  = img_data[i]['label'][j]
                figure['left']   = img_data[i]['left'][j]
                figure['top']    = img_data[i]['top'][j]
                figure['width']  = img_data[i]['width'][j]
                figures.append(figure)
            struct_count += 1
            item['boxes'] = figures
            result.append(item)
        return result


In [0]:
with zipfile.ZipFile('/content/drive/My Drive/Data Science/SVHN/street-view-house-numbers.zip', 'r') as zip_file:
  zip_file.extractall('')

In [0]:
#train_folders = 'train'
#test_folders = 'test'

dsf = DigitStructFile('/content/train_digitStruct.mat')
train_data = dsf.get_all_DigitStructure_ByDigit()

print('Train ready')


dsf = DigitStructFile('/content/test_digitStruct.mat')
test_data = dsf.get_all_DigitStructure_ByDigit()

print('Test ready')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


Train ready
Test ready


In [0]:

pickle_file = 'train_data.pickle'

f = open(pickle_file, 'wb', )
pickle.dump(train_data, f, pickle.HIGHEST_PROTOCOL)


pickle_file = 'test_data.pickle'

f = open(pickle_file, 'wb')
pickle.dump(test_data, f, pickle.HIGHEST_PROTOCOL)


In [0]:
pickle_name = 'train_data.pickle'

with open(pickle_name, 'rb') as f:
    train_data = pickle.load(f)
    
pickle_name = 'test_data.pickle'

with open(pickle_name, 'rb') as f:
    test_data = pickle.load(f)

In [0]:
train_data[0]

{'boxes': [{'height': 219.0,
   'label': 1.0,
   'left': 246.0,
   'top': 77.0,
   'width': 81.0},
  {'height': 219.0, 'label': 9.0, 'left': 323.0, 'top': 81.0, 'width': 96.0}],
 'filename': '1.png'}

In [0]:
print("Length of train data :", len(train_data))
print("Length of test data :", len(test_data))

Length of train data : 33402
Length of test data : 13068


In [0]:
from PIL import Image

images = [Image.open('/content/train/'+train_data[0]['filename'])]

In [0]:
images_name = train_data[0]['filename']
print('/content/train/'+ images_name)

/content/train/1.png


In [0]:
# Show sample images

from PIL import Image

images = [Image.open(os.path.join('train',train_data[j]['filename'])) for j in range(10)]

labels = ['']*10
for j in range(10):
    for i in range(len(train_data[j]['boxes'])):
        labels[j] += str(int(train_data[j]['boxes'][i]['label']))

plt.figure(figsize=(40,20))
columns = 5
for i, image in enumerate(images):
    plt.subplot(len(images) / columns + 1, columns, i + 1)
    plt.title('Label: '+labels[i], fontsize=30)
    plt.xlabel('Size: '+str(image.size), fontsize=30)
    plt.imshow(image)